# Information Retrieval Lab WiSe 2024/2025: Baseline Retrieval System

This Jupyter notebook serves as a baseline retrieval system that you can improve upon.
We use subsets of the MS MARCO datasets to retrieve passages of web documents.
We will show you how to create a software submission to TIRA from this notebook.

An overview of all corpora that we use in the current course is available at [https://tira.io/datasets?query=ir-lab-wise-2024](https://tira.io/datasets?query=ir-lab-wise-2024). The dataset IDs for loading the datasets are:

- `ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training`: A subsample of the TREC 2019/2020 Deep Learning tracks on the MS MARCO v1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/subsampled-ms-marco-rag-20241202-training` (_work in progress_): A subsample of the TREC 2024 Retrieval-Augmented Generation track on the MS MARCO v2.1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/ms-marco-rag-20241203-test` (work in progress): The test corpus that we have created together in the course, based on the MS MARCO v2.1 passage dataset. We will use this dataset as the test dataset, i.e., evaluation scores become available only after the submission deadline.

### Step 1: Import libraries

We will use [tira](https://tira.io/), an information retrieval shared task platform, and [ir_dataset](https://ir-datasets.com/) for loading the datasets. Subsequently, we will build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine framework.

First, we need to install the required libraries.

In [1]:
!pip3 install 'tira>=0.0.139' ir-datasets 'python-terrier==0.10.0' 
!pip3 install --upgrade pyterrier-caching pyterrier_t5
# !pip3 install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

  Using cached python_terrier-0.10.0-py3-none-any.whl
  Attempting uninstall: python-terrier
    Found existing installation: python-terrier 0.12.0
    Uninstalling python-terrier-0.12.0:
      Successfully uninstalled python-terrier-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyterrier-caching 0.2.2 requires python-terrier>=0.11.0, but you have python-terrier 0.10.0 which is incompatible.
  Using cached python_terrier-0.12.0-py3-none-any.whl.metadata (11 kB)
Using cached python_terrier-0.12.0-py3-none-any.whl (145 kB)
  Attempting uninstall: python-terrier
    Found existing installation: python-terrier 0.10.0
    Uninstalling python-terrier-0.10.0:
      Successfully uninstalled python-terrier-0.10.0


Create an API client to interact with the TIRA platform (e.g., to load datasets and submit runs).

In [ ]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

terrier-assemblies 5.10 jar-with-dependencies not found, downloading to /Users/till/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /Users/till/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
/var/folders/2z/hftmcxnx6857z5y2jcyv934h0000gn/T/ipykernel_43600/3276418669.py:5: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


### Step 2: Load the dataset

We load the dataset by its ir_datasets ID (as listed in the Readme). Just be sure to add the `irds:` prefix before the dataset ID to tell PyTerrier to load the data from ir_datasets.

In [3]:
from pyterrier import get_dataset

pt_dataset = get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')

### Step 3: Build an index

We will then create an index from the documents in the dataset we just loaded.

In [4]:
from pyterrier import IterDictIndexer

indexer = IterDictIndexer(
    # Store the index in the `index` directory.
    "../data/index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)
index = indexer.index(pt_dataset.get_corpus_iter())

Download from Zenodo: https://zenodo.org/records/14254044/files/subsampled-ms-marco-deep-learning-20241201-training-inputs.zip


Download: 100%|██████████| 9.51M/9.51M [00:01<00:00, 7.31MiB/s]


Download finished. Extract...
Extraction finished:  /Users/till/.tira/extracted_datasets/ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training/


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 26034/68261 [00:04<00:05, 7602.78it/s]

08:27:07.440 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:08<00:00, 7605.01it/s] 


08:27:12.966 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


### Step 4: Define the retrieval pipeline

We will define a simple retrieval pipeline using just BM25 as a baseline. For details, refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io) or [tutorial](https://github.com/terrier-org/ecir2021tutorial).

In [5]:
from pyterrier import BatchRetrieve

bm25 = BatchRetrieve(index, wmodel="BM25")

/var/folders/2z/hftmcxnx6857z5y2jcyv934h0000gn/T/ipykernel_43600/3803264467.py:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = BatchRetrieve(index, wmodel="BM25")


### Step 5: Create the run
In the next steps, we would like to apply our retrieval system to some topics, to prepare a 'run' file, containing the retrieved documents.

First, let's have a short look at the first three topics:

In [6]:
# The `'text'` argument below selects the topics `text` field as the query.
pt_dataset.get_topics('text')

Download from Zenodo: https://zenodo.org/records/14254044/files/subsampled-ms-marco-deep-learning-20241201-training-truths.zip


Download: 100%|██████████| 61.7k/61.7k [00:00<00:00, 807kiB/s]

Download finished. Extract...
Extraction finished:  /Users/till/.tira/extracted_datasets/ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training/


,qid,query
0,1030303,who is aziz hashim
1,1037496,who is rep scalise
2,1043135,who killed nicholas ii of russia
3,1051399,who sings monk theme song
4,1064670,why do hunters pattern their shotguns
...,...,...
92,405717,is cdg airport in main paris
93,182539,example of monotonic function
94,1113437,what is physical description of spruce
95,1129237,hydrogen is a liquid below what temperature


Now, retrieve results for all the topics (may take a while):

In [8]:
run = bm25(pt_dataset.get_topics('text'))

That's it for the retrieval. Here are the first 10 entries of the run:

In [9]:
run.head(10)

,qid,docid,docno,rank,score,query
0,1030303,53852,8726436,0,31.681671,who is aziz hashim
1,1030303,56041,8726433,1,25.966276,who is aziz hashim
2,1030303,62116,8726435,2,23.863442,who is aziz hashim
3,1030303,32183,8726429,3,23.391821,who is aziz hashim
4,1030303,35867,8726437,4,21.030669,who is aziz hashim
5,1030303,17637,8726430,5,19.967200,who is aziz hashim
6,1030303,42957,7156982,6,19.967200,who is aziz hashim
7,1030303,21803,8726434,7,19.474804,who is aziz hashim
8,1030303,59828,1305520,8,17.849161,who is aziz hashim
9,1030303,60002,3302257,9,17.832781,who is aziz hashim


### Step 6: Persist and upload run to TIRA

The output of our retrieval system is a run file. This run file can later (and, e.g., in a different notebook or by a different person) be statistically evaluated. We will therefore first upload the run to TIRA.

In [10]:
from tira.third_party_integrations import persist_and_normalize_run

persist_and_normalize_run(
    run,
    # Give your approach a short but descriptive name tag.
    system_name='bm25-baseline', 
    default_output='../data/runs',
    upload_to_tira=pt_dataset,
)

The run file is normalized outside the TIRA sandbox, I will store it at "../data/runs".
Done. run file is stored under "../data/runs/run.txt.gz".
Run uploaded to TIRA. Claim ownership via: https://www.tira.io/claim-submission/cb11d8af-4ca2-49de-bdfa-6ab8a83b1c1d


Click on the link in the cell output above to claim your submission on TIRA.

# Step 7: Improve

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

Ideen:
- Lemmatisierung/Stemming andere austesten
- Textembeddings (Transformer): Bi-Encoder
- Reranking (Top10 in LLM reingeben): monoT5 (das hier machen)
- Hyperparameter-Tuning: BM25 Parameter (b, k1, k2) anpassen (Grid-Search)
- Query-Pipeline anpassen (Stoppwörter ergänzen/austauschen, usw.) + Query Expansion: Synonyme zur Query hinzufügen
- Document Expansion: doc2query
- verschiedene Retrieval Ansätze verknüpfen: Learning-to-Rank

In [7]:
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker
monoT5 = MonoT5ReRanker()
duoT5 = DuoT5ReRanker()

/opt/homebrew/Caskroom/miniconda/base/envs/ir/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
mono_pipeline = bm25 % 100 >> pt.text.get_text(pt_dataset, "text") >> monoT5
duo_pipeline = mono_pipeline % 10 >> duoT5

In [ ]:
# Experiment
from pyterrier import Experiment
# Anzahl der Topics
first_x_topics = 1

Experiment(
    [bm25, mono_pipeline, duo_pipeline],
    topics = pt_dataset.get_topics('text').head(first_x_topics),
    qrels = pt_dataset.get_qrels(),
    eval_metrics =['ndcg_cut_10'],
    names=["BM25", "monoT5", "duoT5"],
    round = 3
)

duoT5: 100%|██████████| 1/1 [00:25<00:00, 25.95s/queries]


,name,ndcg_cut_10
0,BM25,0.873
1,monoT5,0.627
2,duoT5,0.758


# Caching

In [34]:
from pyterrier_caching import SparseScorerCache

first_x_topics = 100

# (bm25 % 100) damit nur die ersten 100 Dokumente genommen werden
inp = (bm25 % 400).transform(pt_dataset.get_topics('text').head(first_x_topics)) # BM25 auf den ersten x Topics anwenden. Dabei cutoff von 100 Dokumenten
mono2 = pt.text.get_text(pt_dataset, "text") >> monoT5 # Pipeline für MonoT5
mono_cached = SparseScorerCache('monoT5.cache', mono2, verbose=True) # Caching für MonoT5
duo2 = pt.text.get_text(pt_dataset, "text") >> duoT5 # Pipeline für DuoT5
duo_cached = SparseScorerCache('duoT5.cache', duo2, verbose=True) # Caching für DuoT5

mono_results = mono_cached.transform(inp) # MonoT5 wird auf die Ergebnisse von BM25 angewendet
mono_results_10 = (mono_cached % 10).transform(inp) # Nur die Top10 Dokumente, damit der Input zu DuoT5 gegeben werden kann

duo_results = duo_cached.transform(mono_results_10) # DuoT5 wird auf die Ergebnisse von MonoT5 angewendet

Sqlite3ScorerCache('monoT5.cache', (<pyterrier.datasets.IRDSTextLoader object at 0x33047da20> >> MonoT5(castorini/monot5-base-msmarco)), group='query', key='docno'): 37627 hit(s), 0 miss(es)
Sqlite3ScorerCache('monoT5.cache', (<pyterrier.datasets.IRDSTextLoader object at 0x33047da20> >> MonoT5(castorini/monot5-base-msmarco)), group='query', key='docno'): 37627 hit(s), 0 miss(es)
Sqlite3ScorerCache('duoT5.cache', (<pyterrier.datasets.IRDSTextLoader object at 0x1479fb8b0> >> DuoT5(castorini/duot5-base-msmarco)), group='query', key='docno'): 965 hit(s), 0 miss(es)


In [35]:
# Experiment wird ausgeführt mit den berechneten DataFrames
Experiment(
    [bm25, mono_results, duo_results],
    topics = pt_dataset.get_topics('text').head(first_x_topics),
    qrels = pt_dataset.get_qrels(),
    eval_metrics =['ndcg_cut_10'],
    names=["BM25", "monoT5", "duoT5"],
    round = 3
)

,name,ndcg_cut_10
0,BM25,0.489
1,monoT5,0.724
2,duoT5,0.728
